<a href="https://colab.research.google.com/github/leman-cap13/my_projects/blob/main/Airline_Passenger_Satisfaction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
files.upload()

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download teejmahal20/airline-passenger-satisfaction

In [ ]:
import zipfile
zip_ref = zipfile.ZipFile('/content/airline-passenger-satisfaction.zip', 'r')
zip_ref.extractall()


In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)

In [ ]:
df=pd.read_csv('/content/train.csv')
df

In [ ]:
df.isna().sum()

In [ ]:
df.describe()

In [ ]:
df.dtypes

In [ ]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
df['satisfaction']=le.fit_transform(df['satisfaction'])
df.head()

In [ ]:
df.corr(numeric_only=True)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
plt.figure(figsize=(12, 8))
sns.heatmap(df.corr(numeric_only=True), annot=True, fmt=".2f", cmap="coolwarm")
plt.show()

In [ ]:
df.columns

In [ ]:
df.drop('id', inplace=True, axis=1)

In [ ]:
df.drop('Unnamed: 0', inplace=True, axis=1)

In [ ]:
df.head()

In [ ]:
from sklearn.model_selection import train_test_split
X=df.drop('satisfaction', axis=1)
y=df['satisfaction'].copy()

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC


In [ ]:
num_feature=X_train.select_dtypes(include=[np.number]).columns
cat_feature=X_train.select_dtypes(exclude=[np.number]).columns

In [ ]:
num_pipeline=Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

cat_pipeline=Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent', fill_value='missing')),
    ('encoder', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

transformer=ColumnTransformer([
    ('num', num_pipeline, num_feature),
    ('cat', cat_pipeline, cat_feature)
], remainder='passthrough')

estimator=RandomForestClassifier(random_state=42)

full_pipeline=Pipeline([
    ('preprocessing', transformer),
    ('estimator', estimator)
])


In [ ]:
full_pipeline.fit(X_train, y_train)

In [ ]:
full_pipeline.score(X_train, y_train),full_pipeline.score(X_test, y_test)

In [ ]:
full_pipeline.predict(X_test[:10])

In [ ]:
preprocessor = full_pipeline.named_steps['preprocessing']


feature_names = preprocessor.get_feature_names_out()


importances = full_pipeline.named_steps['estimator'].feature_importances_

feat_importance = pd.DataFrame({
    'Feature': feature_names,
    'Importance': importances
}).sort_values(by='Importance', ascending=False)


plt.figure(figsize=(10, 6))
plt.barh(feat_importance['Feature'], feat_importance['Importance'])
plt.gca().invert_yaxis()
plt.xlabel("Importance")
plt.title("Feature Importance")
plt.show()


In [ ]:
y_test[:10]

In [ ]:
y_pred=full_pipeline.predict(X_test)

In [ ]:
accuracy_score(y_test, y_pred)

In [ ]:
precision_score(y_test, y_pred)

In [ ]:
recall_score(y_test, y_pred)

In [ ]:
f1_score(y_test, y_pred)

In [ ]:
cm=confusion_matrix(y_test, y_pred)
cm

In [ ]:
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

#Grid SearchCV

In [ ]:
param_grid = {
    'estimator__n_estimators': [50, 100, 200],
    'estimator__max_depth': [10,20,30],
    'estimator__min_samples_split': [2, 5, 10],
    'estimator__min_samples_leaf': [1, 2, 4]
}

grid_search = GridSearchCV(full_pipeline, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train[:2000], y_train[:2000])

In [ ]:
grid_search.best_params_

In [ ]:
grid_search.best_score_

In [ ]:
best_model = grid_search.best_estimator_


#Logistic Regression

In [ ]:
estimator2=LogisticRegression(random_state=42)

full_pipeline_2=Pipeline([
    ('preprocessing', transformer),
    ('estimator2', estimator2)
])

In [ ]:
full_pipeline_2.fit(X_train, y_train)

In [ ]:
full_pipeline_2.score(X_train, y_train),full_pipeline_2.score(X_test, y_test)

In [ ]:
y_prediction=full_pipeline_2.predict(X_test)

In [ ]:
accuracy_score(y_test, y_prediction)

In [ ]:
precision_score(y_test, y_prediction)

In [ ]:
recall_score(y_test, y_prediction)

In [ ]:
f1_score(y_test, y_prediction)

In [ ]:
cm=confusion_matrix(y_test, y_prediction)
cm

In [ ]:
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

#Support Vector Classification

In [ ]:
estimator3=SVC(random_state=42)

full_pipeline_3=Pipeline([
    ('preprocessing', transformer),
    ('estimator3', estimator3)
])

In [ ]:
full_pipeline_3.fit(X_train, y_train)

In [ ]:
full_pipeline_3.score(X_train, y_train),full_pipeline_3.score(X_test, y_test)